# Notebook 2 - Defects / Features

This notebook demonstrates the library of different defects than can be introduced to a pipe in PipeFactory.

First we build a pipe standard pipe, this is a straight pipe of length 350.0 m and has a radius of 70.0 and thickness of 3.0.

In [1]:
import numpy as np
import pipefactory as pf

Straight0 = {
    'Name': 'Straight0',
    'length': 2.0,
    'type': 'Straight',
}

section_list = [Straight0]

mesh = pf.Pipe(outer_radius=0.0365, 
               thickness = 0.00305, 
               section_list=section_list, 
               elem_type=("hex", False), 
               element_size = 0.01, 
               elements_through_thickness = 2,
               element_around_circum=32)

mesh.export('foo.vtk')


ImportError: attempted relative import beyond top-level package

There are two broad classes of defects that have been implemented. These involved one of two things

1. Displacing nodes, which include Dimple and Weld defects.

2. Removing elements, which includes involves holes and slits.

We will go through all types.

## Dimple Defects

These are defects which represent a dent (inwards). To define them we define a number of parameters

- $s_0$ (m) the location along the length of the midline of the pipe.

- $\phi_0$ (degrees) the radial angle of the centre of the dimple.

- $A$ (m) the amplitude of the dimple.

- $\ell$ (m) lengthscale parameter controlling the extend of the dimple.

In [2]:
mesh = pf.Pipe(outer_radius=0.0365, 
               thickness = 0.00305, 
               section_list=section_list, 
               elem_type=("hex", False), 
               element_size = 0.01, 
               elements_through_thickness = 2,
               element_around_circum=32)

s0 = 1.0
phi0 = np.pi
A = 0.01 
ell = 0.06

mesh.add_defect_displacement(pf.Dimple(s0, phi0, A, ell))

mesh.export('foo_dimple.vtk')

Multiple defects can be added by simple calling the method '.add_defect_displacement()' more than once. Like this

In [3]:
mesh.add_defect_displacement(pf.Dimple(0.5, np.pi/2, A, ell))

mesh.export('foo_multiple_defects.vtk')

## Weld

The next defect / feature defines a weld as a thickening around the complete collar of the pipe. It is defined by

- $s_0$ (m) the location along the length of the midline of the pipe.

- $A_{out}$ (degrees)  the amplitude of the dimple on the outside

- $A_{in}$ (m) the amplitude of the dimple on the inside

- $r$ (m) radius of the pipe

- $t$ (m) thickness of the pipe

In [4]:
s0 = 1.0
Aout = 0.04
Ain = 0.02
ell_out = 0.08
ell_in = 0.04

mesh_weld =  pf.Pipe(outer_radius=0.0365, 
               thickness = 0.00305, 
               section_list=section_list, 
               elem_type=("hex", False), 
               element_size = 0.01, 
               elements_through_thickness = 2,
               element_around_circum=32)

mesh_weld.add_defect_displacement(pf.Weld(s0, Aout, Ain, ell_out, ell_in))

mesh_weld.export('foo_weld.vtk')

## Hole

To introduce a circular hole we called this time the method 'remove_elements()'. A hole is defed by three variables

- $s_0$ (m) the location along the length of the midline of the pipe.

- $\phi_0$ (degrees) the radial angle of the centre of the dimple.

- $r$ (m) the radius of the hole.

We start by building a fresh mesh. Then make and visualise the hole.

In [5]:
s0 = 1.0
phi0 = np.pi/4
radius = 0.05

mesh_with_hole = pf.Pipe(outer_radius=0.0365, 
               thickness = 0.00305, 
               section_list=section_list, 
               elem_type=("hex", False), 
               element_size = 0.01, 
               elements_through_thickness = 2,
               element_around_circum=32)

mesh_with_hole.remove_elements(pf.Hole(s0, phi0, radius))

mesh_with_hole.export('foo_hole.vtk')

## Slits

In a simple way to a whole we can introduce a slit. The difference with this is we have introduced a slit profile. Which allows you to define the profile of the slit, so that not all elements are removed through thickness.

Naturally this collectively has more parameters to define the "slit".

- $s_0$ (m) the location along the length of the midline of the pipe.

- $\phi_0$ (degrees) is the angle at the start of the slit

- $\phi_1$ (degrees) is the angle at the end of the slit

- $w$ (m) is the slit width.

- 'partial' indicates if there is only a partial slit, i.e. not all elements through thickness are removed.

- 'profile' is a user-defined function which describes the profile of the slit through the thickness fo the slit.

So Let's do a simple slit first, for which all elements are removed.

In [6]:
mesh_with_slit = pf.Pipe(outer_radius=0.0365, 
               thickness = 0.00305, 
               section_list=section_list, 
               elem_type=("hex", False), 
               element_size = 0.01, 
               elements_through_thickness = 2,
               element_around_circum=32)

mesh_with_slit.remove_elements(pf.Radial_Slit(s0=1.0, phi0=0., phi1=np.pi/4, slit_width=0.03, partial = False))

mesh_with_slit.export('foo_slit.vtk')

We can now build the same slit which has a partial profile in it.

In [7]:
mesh_with_slit = pf.Pipe(outer_radius=0.0365, 
               thickness = 0.00305, 
               section_list=section_list, 
               elem_type=("hex", False), 
               element_size = 0.01, 
               elements_through_thickness = 2,
               element_around_circum=32)

class myprofile:
    def __init__(self):
        self.A = 1.8
    def __call__(self, ds, z):
        if z > 1. - self.A * np.exp(-12.0*(ds-0.5)**2):
            return True
        else:
            return False

mesh_with_slit.remove_elements(pf.Radial_Slit(s0=1.0, phi0=0., phi1=np.pi/4, slit_width=0.03, partial = True, profile=myprofile()))

mesh_with_slit.export('foo_partial_slit.vtk')

## Cracks

In order to simulate thin cracks without the need for axial refinement, we must create degenerate elements that not connected to adjacent nodes. This gives the ability to make cracks going from very thin to zero volume.

`RadialCrack` creates a radial rectangular slit with the following parameters:

- $s_0$ (m) the location along the length of the midline of the pipe.

- $\phi_0$ (degrees) is the angle at the start of the crack

- $\phi_1$ (degrees) is the angle at the end of the crack

- 'crack_width' (m) is the crack width (**Important** must be less the local element length)

- 'crack_depth' (m) is the depth of the crack (can be all the way through).

- 'smoothing_dist' (m) is the distance from the crack to smooth deformation of elements through the thickness.

In [10]:
import numpy as np
import pipefactory as pf

Straight0 = {
    'Name': 'Straight0',
    'length': 2.0,
    'type': 'Straight',
}

section_list = [Straight0]

mesh = pf.Pipe(outer_radius=0.0365, 
               thickness = 0.01, 
               section_list=section_list, 
               elem_type=("hex", False), 
               element_size = 0.01, 
               elements_through_thickness = 2,
               element_around_circum=48)

mesh.degenerate_crack(pf.RadialCrack(s0=0.5,phi0=0.1,phi1=2.0,crack_width=0.001,crack_depth=0.011,smoothing_dist=0.03))

mesh.export("foo_crack.vtk")